In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [5]:

ratings_count = df_ratings['rating'].value_counts()
df_ratings = df_ratings[df_ratings['rating'] > 0]
df_ratings = df_ratings[df_ratings['user'].isin(df_ratings['user'].value_counts()[df_ratings['user'].value_counts() >= 200].index)]
df_ratings = df_ratings[df_ratings['isbn'].isin(df_ratings['isbn'].value_counts()[df_ratings['isbn'].value_counts() >= 100].index)]


pivot_table = df_ratings.pivot(index='user', columns='isbn', values='rating').fillna(0)

matrix = csr_matrix(pivot_table.values)

title_to_index = {title: i for i, title in enumerate(df_books['title'])}

knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(matrix)


NameError: name 'df_ratings' is not defined

In [4]:
# function to return recommended books - this will be tested

def get_recommends(book_title):
    if book_title not in title_to_index:
        return f"Book with title '{book_title}' not found in the dataset."

    book_index = title_to_index[book_title]
    distances, indices = knn_model.kneighbors(matrix[book_index], n_neighbors=6)

    recommended_books = [book_title, []]
    for i in range(1, len(distances.flatten())):
        recommended_books[1].append([df_books['title'][indices.flatten()[i]], distances.flatten()[i]])

    return recommended_books

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()